In [5]:
import numpy as np
import pandas as pd

embeddings_names = [
    "polblogs",
    "cs_phds",
    "cora",
    "citeseer",
]

# Copied from notebook 2
SIGNATURES = [
    [(-1, 2), (-1, 2)],  # HH
    [(-1, 2), (0, 2)],  # HE
    [(-1, 2), (1, 2)],  # HS
    [(1, 2), (1, 2)],  # SS
    [(1, 2), (0, 2)],  # SE
    [(-1, 4)],  # H
    [(0, 4)],  # E
    [(1, 4)],  # S
]
SIGNATURES_STR = ["HH", "HE", "HS", "SS", "SE", "H", "E", "S"]
n_trials = 10

bad = []
for embedding in embeddings_names:
    for signature in SIGNATURES_STR:
        for trial in range(n_trials):
            my_data = np.load(f"../data/graphs/embeddings/{embedding}/{signature}_{trial}.npy")
            if np.isnan(my_data).any():
                bad.append((embedding, signature, trial))
                print(embedding, signature, trial)
print(bad)

[]


In [20]:
dists

,Unnamed: 0,signature,seed,d_avg
0,0,"[(-1, 2), (-1, 2)]",0,0.187309
1,1,"[(-1, 2), (-1, 2)]",1,0.183700
2,2,"[(-1, 2), (-1, 2)]",2,0.188372
3,3,"[(-1, 2), (-1, 2)]",3,0.185585
4,4,"[(-1, 2), (-1, 2)]",4,0.182979
...,...,...,...,...
75,75,"[(1, 4)]",75,0.183932
76,76,"[(1, 4)]",76,0.183157
77,77,"[(1, 4)]",77,0.179597
78,78,"[(1, 4)]",78,0.183023


In [25]:
import embedders
import pandas as pd
from tqdm.notebook import tqdm

MAX_DEPTH = 3
# MAX_DEPTH = None
N_FEATURES = "d_choose_2"

results = []
my_tqdm = tqdm(total=len(embeddings_names) * n_trials * len(SIGNATURES))
for embedding in embeddings_names:
    _, y, _ = embedders.dataloaders.load(embedding)
    dists = pd.read_table(f"../data/graphs/embeddings/{embedding}_dists.tsv")
    for sig, sig_str in zip(SIGNATURES, SIGNATURES_STR):
        pm = embedders.manifolds.ProductManifold(signature=sig)
        dists_sig = dists[dists["signature"] == str(sig)]

        for i, trial in enumerate(dists_sig["seed"]):
            # i is used to load the correct embedding
            # trial is used to get the correct d_avg
            # I'm sorry for this mess
            X = np.load(f"../data/graphs/embeddings/{embedding}/{sig_str}_{i}.npy")

            res = embedders.benchmarks.benchmark(
                X,
                y,
                pm,
                # models=["sklearn_dt", "product_dt"],
                max_depth=MAX_DEPTH,
                n_features=N_FEATURES,
            )
            res["embedding"] = embedding
            res["signature"] = sig_str
            res["trial"] = trial

            # Get matching d_avg using seed and sig_str
            dists_seed = dists_sig[dists_sig["seed"] == trial]
            assert len(dists_seed) == 1
            res["d_avg"] = dists_seed["d_avg"].values[0]

            results.append(res)
            my_tqdm.update(1)

results = pd.DataFrame(results)

  0%|          | 0/320 [00:00<?, ?it/s]

Top CC has 1222 nodes; original graph has 1490 nodes.
Top CC has 1025 nodes; original graph has 1025 nodes.
Top CC has 2485 nodes; original graph has 2708 nodes.
Top CC has 2110 nodes; original graph has 3264 nodes.


In [26]:
# results.to_csv("../data/results/graphs.tsv", sep="\t", index=False)
results.to_csv(f"../data/results/graphs_dC2_md{MAX_DEPTH}.tsv", sep="\t", index=False)

In [23]:
results.groupby(["embedding", "signature"]).mean()

sklearn_dt  sklearn_rf  product_dt  product_rf  \
embedding signature                                                   
citeseer  E            0.224645    0.246919    0.231043    0.251185   
          H            0.224171    0.236019    0.216351    0.249526   
          HE           0.224408    0.244076    0.232464    0.236019   
          HH           0.223223    0.241469    0.221564    0.237678   
          HS           0.228199    0.247867    0.227014    0.242891   
          S            0.227962    0.231517    0.235308    0.235071   
          SE           0.226777    0.229621    0.222275    0.234834   
          SS           0.221090    0.244787    0.230095    0.238152   
cora      E            0.193763    0.208048    0.196579    0.209859   
          H            0.188732    0.199396    0.182495    0.200805   
          HE           0.192555    0.201207    0.204024    0.214487   
          HH           0.196781    0.212072    0.191952    0.213883   
          HS           0.191348    0.218511    0.194769    0.215292   
          S            0.190342    0.198994    0.189336    0.202616   
          SE           0.194366    0.213280    0.184306    0.216097   
          SS           0.184909    0.206237    0.178270    0.209859   
cs_phds   E            0.071220    0.063902    0.067805    0.070732   
          H            0.076585    0.076585    0.072195    0.075610   
          HE           0.077561    0.080000    0.063902    0.073659   
          HH           0.074797    0.083469    0.073713    0.075339   
          HS           0.060000    0.066341    0.060000    0.065854   
          S            0.061951    0.063415    0.060976    0.071220   
          SE           0.061951    0.069268    0.062439    0.067317   
          SS           0.058049    0.076098    0.062439    0.068780   
polblogs  E            0.905714    0.931020    0.909796    0.936735   
          H            0.914286    0.937143    0.908163    0.935918   
          HE           0.903265    0.929796    0.913878    0.933878   
          HH           0.899592    0.929388    0.908980    0.924490   
          HS           0.911837    0.933061    0.904490    0.935102   
          S            0.917551    0.933878    0.915510    0.928163   
          SE           0.915102    0.930612    0.904082    0.929388   
          SS           0.908571    0.937143    0.922041    0.935102   

                     tangent_dt  tangent_rf       knn  ps_perceptron  trial  \
embedding signature                                                           
citeseer  E            0.221801    0.240047  0.246445       0.136256    4.5   
          H            0.222986    0.240284  0.251896       0.138626    4.5   
          HE           0.225829    0.246682  0.240995       0.139573    4.5   
          HH           0.214455    0.245972  0.254502       0.055213    4.5   
          HS           0.224408    0.243365  0.260900       0.132227    4.5   
          S            0.230569    0.240047  0.243365       0.138152   18.5   
          SE           0.218246    0.237915  0.246682       0.132701    4.5   
          SS           0.231280    0.235545  0.253318       0.139336    4.5   
cora      E            0.188330    0.207243  0.206439       0.172837   64.5   
          H            0.176056    0.198994  0.207042       0.181087   54.5   
          HE           0.195976    0.208249  0.218712       0.175855   14.5   
          HH           0.187726    0.207847  0.204628       0.051509    4.5   
          HS           0.190141    0.208451  0.220322       0.163380   24.5   
          S            0.189336    0.210463  0.212877       0.164588   74.5   
          SE           0.197586    0.207243  0.207445       0.184708   44.5   
          SS           0.188330    0.210060  0.207646       0.201408   34.5   
cs_phds   E            0.064878    0.064390  0.057561       0.039512   64.5   
          H            0.067805    0.079512  0.065366       0.049268   54.5   
          HE           0.0